In [3]:
from pyspark.sql import SparkSession

# Create a SparkSession
spark = SparkSession.builder.appName("Stocks").getOrCreate()

# Load the data into a DataFrame
df = spark.read.csv("/content/drive/MyDrive/Big data/financial_data_2000_2018_cleaned.csv", header=True, inferSchema=True)


In [4]:
from pyspark.sql import functions
from pyspark.sql.types import *

# create the Date and year columns
df = df.withColumn('Date', functions.from_unixtime('timestamp').cast(DateType()))
df = df.withColumn("Year", df["Date"].substr(1, 4))

In [5]:
df.show()

+---+-----+----------+----+----+----+---------+-------+----------+----+
|_c0|stock| timestamp|open|high| low|adj_close| volume|      Date|Year|
+---+-----+----------+----+----+----+---------+-------+----------+----+
|  0| GERN|1532988000|3.34|3.59|3.59|     3.59|4650479|2018-07-30|2018|
|  1| GERN|1532901600|3.36|3.45|3.45|      3.3|3571600|2018-07-29|2018|
|  2| GERN|1532642400|3.44| 3.5| 3.5|     3.34|2928697|2018-07-26|2018|
|  3| GERN|1532556000|3.26|3.56|3.56|     3.44|4209642|2018-07-25|2018|
|  4| GERN|1532469600|3.37|3.42|3.42|     3.22|3568900|2018-07-24|2018|
|  5| GERN|1532383200|3.41|3.45|3.45|     3.35|3323260|2018-07-23|2018|
|  6| GERN|1532296800|3.48|3.49|3.49|     3.37|2339213|2018-07-22|2018|
|  7| GERN|1532037600|3.45|3.46|3.46|     3.43|7744407|2018-07-19|2018|
|  8| GERN|1531951200|3.59| 3.6| 3.6|     3.48|5923377|2018-07-18|2018|
|  9| GERN|1531864800|3.57|3.63|3.63|     3.59|3021013|2018-07-17|2018|
| 10| GERN|1531778400| 3.6|3.63|3.63|     3.59|4404628|2018-07-1

### Question 1

In [6]:
# Count the number of distinct stocks
num_stocks = df.select("stock").distinct().count()

# Print the number of stocks
print(f"The number of distinct stocks in the dataset is: {num_stocks}")

The number of distinct stocks in the dataset is: 39


### Question 2

In [7]:
# Filter the DataFrame to include only rows where the "Open" column is greater than 60
# group the data by the "Date" column
num_days = df.filter(df["open"] > 60).select("timestamp").distinct().count()

# The number of days
print(f"The number of days where the 'open' indicator was above 60 dollars is: {num_days}")

The number of days where the 'open' indicator was above 60 dollars is: 3135


### Question 3

In [8]:
filtered_df = df.filter((df["stock"] == "AAPL") & (df["open"] >= 35) & (df["open"] <= 60))

# Count the number of rows in the filtered DataFrame
num_days = filtered_df.select("Date").count()

# Print the number of days
print(f"The number of days where the 'Open' indicator was between 35 and 60 dollars for stock 'AAPL' is: {num_days}")

The number of days where the 'Open' indicator was between 35 and 60 dollars for stock 'AAPL' is: 484


### Question 4

In [22]:
# Filter the DataFrame to include only rows where the "Low" column is less than 1
filtered_df = df.filter(df.low < 1)

# Group the filtered DataFrame by year and count the number of rows in each group
# We shouldn't count the same stock twice
num_days_per_year_per_day = filtered_df.groupBy(["Year", 'Date']).count()
num_days_per_year = num_days_per_year_per_day.groupBy(["Year"]).count()

# Print the number of days per year
num_days_per_year.show()

+----+-----+
|Year|count|
+----+-----+
|2000|   47|
|2002|  102|
|2009|  252|
|2008|   96|
|2001|    2|
|2010|  114|
|2003|   85|
+----+-----+

